In [3]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import sqlite3
pd.set_option('display.max_columns', None)    

load_dotenv()  # charge les variables d'environnement à partir du fichier .env
api_key = os.getenv('API_KEY')  # récupère la valeur de la variable d'environnement API_KEY

In [4]:
import requests
import pandas as pd

# liste des villes pour lesquelles on souhaite récupérer les informations météorologiques
cities = ['Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes', 'Strasbourg', 'Montpellier', 'Bordeaux', 'Lille']

# URL de l'API OpenWeatherMap
url = 'https://api.openweathermap.org/data/2.5/weather'

# clé API OpenWeatherMap

# initialisation du DataFrame
weather_data = pd.DataFrame(columns=['Ville', 'Température actuelle', 'Température ressentie', 'Température minimale', 'Température maximale', 'Pression atmosphérique', 'Humidité', 'Vitesse du vent', 'Direction du vent', 'Lever du soleil', 'Coucher du soleil'])

# récupération des informations météorologiques pour chaque ville de la liste
for city in cities:
    params = {'q': city, 'appid': api_key, 'units': 'metric', 'lang': 'fr'}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        weather_data = pd.concat([weather_data, pd.DataFrame({
            'Ville': [city],
            'Température actuelle': [data['main']['temp']],
            'Température ressentie': [data['main']['feels_like']],
            'Température minimale': [data['main']['temp_min']],
            'Température maximale': [data['main']['temp_max']],
            'Pression atmosphérique': [data['main']['pressure']],
            'Humidité': [data['main']['humidity']],
            'Vitesse du vent': [data['wind']['speed']],
            'Direction du vent': [data['wind']['deg']],
            'Lever du soleil': [pd.to_datetime(data['sys']['sunrise'], unit='s').strftime('%H:%M:%S')],
            'Coucher du soleil': [pd.to_datetime(data['sys']['sunset'], unit='s').strftime('%H:%M:%S')]
        })])

# afficher la dataframe
weather_data
# export du DataFrame dans un fichier CSV

,Ville,Température actuelle,Température ressentie,Température minimale,Température maximale,Pression atmosphérique,Humidité,Vitesse du vent,Direction du vent,Lever du soleil,Coucher du soleil
0,Paris,10.36,8.86,9.59,11.04,1026,54,7.72,40,05:24:51,18:22:48
0,Marseille,16.92,15.71,13.97,17.59,1011,40,7.15,246,05:16:58,18:05:29
0,Lyon,7.34,6.30,5.75,7.61,1020,85,1.79,88,05:18:42,18:11:05
0,Toulouse,11.92,10.89,10.98,11.99,1021,66,5.66,290,05:32:58,18:21:56
0,Nice,9.00,7.76,8.34,11.65,1012,55,2.33,156,05:09:27,17:58:59
0,Nantes,7.96,4.98,7.81,7.96,1025,76,5.14,60,05:42:06,18:37:00
0,Strasbourg,8.06,4.38,7.14,9.39,1026,53,7.20,70,05:03:34,18:00:56
0,Montpellier,17.12,15.96,16.29,17.50,1014,41,6.17,340,05:23:14,18:12:11
0,Bordeaux,13.15,12.04,13.15,14.63,1022,58,3.60,50,05:40:04,18:31:01
0,Lille,10.53,8.94,8.54,12.19,1028,50,5.66,50,05:20:16,18:21:42


In [2]:
import sqlite3

connexion= sqlite3.connect("bdd.db")
curseur = connexion.cursor()

curseur.execute('''CREATE TABLE IF NOT EXISTS weather_info (
              city TEXT NOT NULL,
              temp FLOAT,
              feels_like FLOAT,
              temp_min FLOAT,
              temp_max FLOAT,
              pressure INT,
              humidity INT,
              wind_speed FLOAT,
              wind_direction FLOAT,
              sunrise TEXT,
              sunset TEXT
              );''')

connexion.commit()

query = "INSERT INTO weather_info (city, temp, feels_like, temp_min, temp_max, pressure, humidity, wind_speed, wind_direction, sunrise, sunset) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
curseur.executemany(query, weather_data.values.tolist())

# Validation de la transaction
connexion.commit()

# Fermeture de la connexion
connexion.close()